# Imports & Data Reading

In [10]:
# Data loading
from google.colab import drive
import zipfile
import pandas as pd
import gdown
import pickle
# Preprocessing
import re
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
stop_words = set(stopwords.words('english'))
# Feature extraction
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.corpora import Dictionary
from gensim.models import TfidfModel, LdaModel


# Util
from tqdm import tqdm
from pprint import pprint


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
drive.mount('/content/drive')

In [103]:
DATA_URL = "https://drive.google.com/file/d/10YvMpIzRSi_GXg91TkrQBL_3JKC5dni7/view?usp=sharing"
PREPROCESSED_ARTICLES_URL = "https://drive.google.com/file/d/1-DQeGkgh-KmOe7uKFEmrj_GCqbKxNJbh/view?usp=sharing"
CORPUS_BOW_URL = "https://drive.google.com/file/d/1-I72t_8jqxIukAdEaxRSBMAYJyfNAT8m/view?usp=sharing"

In [13]:
gdown.download(DATA_URL, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=10YvMpIzRSi_GXg91TkrQBL_3JKC5dni7
To: /content/articles1.csv.zip
100%|██████████| 77.4M/77.4M [00:00<00:00, 203MB/s]


'articles1.csv.zip'

In [14]:
DATA_PATH = "articles1.csv.zip"

In [15]:
# Extract the CSV file from the zip archive
with zipfile.ZipFile(DATA_PATH, "r") as zip_ref:
    zip_ref.extractall()

# Load the CSV data into a Pandas DataFrame
df = pd.read_csv("articles1.csv")

In [16]:
df.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


# Preprocessing

In [17]:
# Extract title and content as its preprocessing will differ from other features
articles =  df['title'] + ' ' + df['content']

In [18]:
articles[0]

'House Republicans Fret About Winning Their Health Care Suit - The New York Times WASHINGTON  —   Congressional Republicans have a new fear when it comes to their    health care lawsuit against the Obama administration: They might win. The incoming Trump administration could choose to no longer defend the executive branch against the suit, which challenges the administration’s authority to spend billions of dollars on health insurance subsidies for   and   Americans, handing House Republicans a big victory on    issues. But a sudden loss of the disputed subsidies could conceivably cause the health care program to implode, leaving millions of people without access to health insurance before Republicans have prepared a replacement. That could lead to chaos in the insurance market and spur a political backlash just as Republicans gain full control of the government. To stave off that outcome, Republicans could find themselves in the awkward position of appropriating huge sums to temporari

In [47]:
def preprocess_articles(articles):

  lemmatizer = WordNetLemmatizer()

  # Remove punctuations and convert text to lowercase
  words = gensim.utils.simple_preprocess(articles, deacc=True)
    
  # Remove stop words & punctuation 
  words = [word for word in words if word not in stop_words and word not in string.punctuation]
  # Lemmatize
  words = [lemmatizer.lemmatize(word) for word in words]

  # remove all non-word and non-space
  # words = re.sub(r'[^\w\s]', '', words) 

  return words


In [ ]:
# tqdm.pandas()
# preprocessed_articles = articles.map(preprocess_articles)

Use below cell to save new preprocessed articles instead of loading already saved ones (don't forget to update `PREPROCESSED_ARTICLES_URL` with the new gdrive link)

In [58]:
# from joblib import Parallel, delayed
# preprocessed_articles = Parallel(n_jobs=4)(delayed(preprocess_articles)(article) for article in tqdm(articles, position=0, leave=True))
# preprocessed_articles = pd.Series(preprocessed_articles)
# pickle.dump(preprocessed_articles, open("preprocessed_articles.pkl", "wb"))
# !cp preprocessed_articles.pkl /content/drive/MyDrive/Edu/NLP

100%|██████████| 50000/50000 [03:00<00:00, 276.76it/s]


In [104]:
gdown.download(PREPROCESSED_ARTICLES_URL, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1-DQeGkgh-KmOe7uKFEmrj_GCqbKxNJbh
To: /content/preprocessed_articles.pkl
100%|██████████| 165M/165M [00:01<00:00, 126MB/s]


'preprocessed_articles.pkl'

In [105]:
preprocessed_articles = pickle.load(open("preprocessed_articles.pkl", "rb"))

In [106]:
preprocessed_articles[0]

['house',
 'republican',
 'fret',
 'winning',
 'health',
 'care',
 'suit',
 'new',
 'york',
 'time',
 'washington',
 'congressional',
 'republican',
 'new',
 'fear',
 'come',
 'health',
 'care',
 'lawsuit',
 'obama',
 'administration',
 'might',
 'win',
 'incoming',
 'trump',
 'administration',
 'could',
 'choose',
 'longer',
 'defend',
 'executive',
 'branch',
 'suit',
 'challenge',
 'administration',
 'authority',
 'spend',
 'billion',
 'dollar',
 'health',
 'insurance',
 'subsidy',
 'american',
 'handing',
 'house',
 'republican',
 'big',
 'victory',
 'issue',
 'sudden',
 'loss',
 'disputed',
 'subsidy',
 'could',
 'conceivably',
 'cause',
 'health',
 'care',
 'program',
 'implode',
 'leaving',
 'million',
 'people',
 'without',
 'access',
 'health',
 'insurance',
 'republican',
 'prepared',
 'replacement',
 'could',
 'lead',
 'chaos',
 'insurance',
 'market',
 'spur',
 'political',
 'backlash',
 'republican',
 'gain',
 'full',
 'control',
 'government',
 'stave',
 'outcome',
 'repu

In [107]:
def extract_features(articles, method='tf-idf'):
    if method == 'tf-idf':
        vectorizer = TfidfVectorizer()
        feature_vectors = vectorizer.fit_transform(articles)
        feature_names = vectorizer.get_feature_names_out()
    elif method == 'countvectorizer':
        vectorizer = CountVectorizer()
        feature_vectors = vectorizer.fit_transform(articles)
        feature_names = vectorizer.get_feature_names_out()
    else:
        raise ValueError('Invalid method specified.')
    # TODO:
      # Add word2vec
    
    return feature_vectors, feature_names

In [108]:
# number of unique tokens
total_tokens = preprocessed_articles.explode().nunique()
print(total_tokens)

152705


In [109]:
# create Dictionary from preprocessed articles
dictionary = Dictionary(preprocessed_articles)

# filter out tokens that appear too much or too little
dictionary.filter_extremes(no_below=10, no_above=0.5, keep_n=150000)

Use below cell to save new corpus bows instead of loading already saved ones (don't forget to update `CORPUS_BOW_URL` with the new gdrive link)

In [110]:
# bag of words corpus from dictionary
#corpus_bow = Parallel(n_jobs=4)(delayed(dictionary.doc2bow)(article) for article in tqdm(preprocessed_articles, position=0, leave=True))
# pickle.dump(corpus_bow, open("corpus_bow.pkl", 'wb'))
# !cp corpus_bow.pkl /content/drive/MyDrive/Edu/NLP

In [111]:
gdown.download(CORPUS_BOW_URL, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1-I72t_8jqxIukAdEaxRSBMAYJyfNAT8m
To: /content/corpus_bow.pkl
100%|██████████| 74.1M/74.1M [00:00<00:00, 208MB/s]


'corpus_bow.pkl'

In [112]:
corpus_bow = pickle.load(open("corpus_bow.pkl", "rb"))

# Modelling

In [113]:
# create TF-IDF Model from corpus
tfidf = TfidfModel(corpus_bow)

# weighted tfidf corpus
corpus_tfidf = tfidf[corpus_bow]

In [ ]:
# Apply LDA with 12 topics
lda_model = LdaModel(corpus=corpus_bow,
                     id2word=dictionary,
                     num_topics=12,
                     passes=10,
                     iterations=100)

In [ ]:
topics = lda_model.show_topics(num_topics=12, num_words=12)
for topic in topics:
    print(topic)

In [ ]:
# Apply LDA with 20 topics
lda_model_20 = LdaModel(corpus=corpus_bow,
                     id2word=dictionary,
                     num_topics=20,
                     passes=15,
                     iterations=100)

In [ ]:
topics = lda_model_20.show_topics(num_topics=20, num_words=15)
for topic in topics:
    print(topic)

In [ ]:
# Apply LDA with TF-IDF and 12 topics
lda_model_tfidf = LdaModel(corpus_tfidf,
                           id2word=dictionary,
                           num_topics=12,
                           passes=10,
                           iterations=100)

In [ ]:
# Print the top 12 topics
topics = lda_model_tfidf.show_topics(num_topics=12, num_words=15)
for topic in topics:
    print(topic)

## Model Evaluation

In [ ]:
from gensim.models import CoherenceModel

In [ ]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=corpus_tfidf, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)